In [1]:
import os
import time

import numpy as np

import cv2
import imageio
import torchvision.transforms as transforms

import torch

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # select device for training, i.e. gpu or cpu

_size = 256, 256
resize = transforms.Resize(_size, interpolation=0)

In [3]:
CHECKPOITS_PATH = "../checkpoints"
TEST_DATASET_PATH = "../data/test/images"
MASK_PATH = "../predictions_test"

# # Set path to test dataset (Like in the instructions)
# CHECKPOITS_PATH = ""
# TEST_DATASET_PATH = "/medico2020"
# MASK_PATH = "/mask"

In [4]:
# Define your model. In this case we're using a basic UNet architecture. The output channel for segmentation should be equal to number of classes we want to segment the image into. For a binary segmentation, this is a value of 1.
class UNet(torch.nn.Module):

    def conv_block(self, channel_in, channel_out):
        return torch.nn.Sequential(
            torch.nn.Conv2d(channel_in, channel_out, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(channel_out),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(channel_out, channel_out, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(channel_out),
            torch.nn.ReLU(inplace=True)
        )


    def __init__(self, channel_in, channel_out, bilinear=None):
        super(UNet, self).__init__()
        self.channel_in = channel_in
        self.channel_out = channel_out
        
        # initial convolutional block
        self.initial = self.conv_block(channel_in, 64)
        
        # encoder layers
        self.down0 = self.conv_block(64, 128)
        self.down1 = self.conv_block(128, 256)
        self.down2 = self.conv_block(256, 512)
        self.down3 = self.conv_block(512, 1024)
        
        # decoder layers
        self.up0_0 = torch.nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.up0_1 = self.conv_block(1024, 512)
        self.up1_0 = torch.nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up1_1 = self.conv_block(512, 256)
        self.up2_0 = torch.nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up2_1 = self.conv_block(256, 128)
        self.up3_0 = torch.nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up3_1 = self.conv_block(128, 64)
        
        # final layer before output
        self.final = torch.nn.Conv2d(64, channel_out, kernel_size=1)

    def forward(self,x):
        "Forward pass"
        x_in= self.initial(x)
        enc0 = self.down0(torch.nn.MaxPool2d(2)(x_in))
        enc1 = self.down1(torch.nn.MaxPool2d(2)(enc0))
        enc2 = self.down2(torch.nn.MaxPool2d(2)(enc1))
        enc3 = self.down3(torch.nn.MaxPool2d(2)(enc2))
        
        dec0 = self.up0_0(enc3)
        diff0 = torch.FloatTensor(list(enc2.size())[2:]) - torch.FloatTensor(list(dec0.shape))[2:]
        dec0 = torch.nn.functional.pad(dec0, (int((diff0/2).floor()[0]), int((diff0/2).ceil()[0]), int((diff0/2).floor()[1]), int((diff0/2).ceil()[1])))
        dec0 = self.up0_1(torch.cat((enc2, dec0), dim=1))

        dec1 = self.up1_0(dec0)
        diff1 = torch.FloatTensor(list(enc1.size())[2:]) - torch.FloatTensor(list(dec1.shape))[2:]
        dec1 = torch.nn.functional.pad(dec1, (int((diff1/2).floor()[0]), int((diff1/2).ceil()[0]), int((diff1/2).floor()[1]), int((diff1/2).ceil()[1])))
        dec1 = self.up1_1(torch.cat((enc1, dec1), dim=1))

        dec2 = self.up2_0(dec1)
        diff2 = torch.FloatTensor(list(enc0.size())[2:]) - torch.FloatTensor(list(dec2.shape))[2:]
        dec2 = torch.nn.functional.pad(dec2, (int((diff2/2).floor()[0]), int((diff2/2).ceil()[0]), int((diff2/2).floor()[1]), int((diff2/2).ceil()[1])))
        dec2 = self.up2_1(torch.cat((enc0, dec2), dim=1))

        dec3 = self.up3_0(dec2)
        diff3 = torch.FloatTensor(list(x.size())[2:]) - torch.FloatTensor(list(dec3.shape))[2:]
        dec3 = torch.nn.functional.pad(dec3, (int((diff3/2).floor()[0]), int((diff3/2).ceil()[0]), int((diff3/2).floor()[1]), int((diff3/2).ceil()[1])))
        dec3 = self.up3_1(torch.cat((x_in, dec3), dim=1))
        
        x_out = self.final(dec3) # ? no activation here
        return x_out

In [5]:
model_name = 'UNet_IoULoss_baseline'
chpt_file_name = "ckpt_" + model_name + ".pth"

# Instantiate the model
model = UNet(channel_in=3, channel_out=1)
model = model.to(DEVICE) # load model to DEVICE


# load best weights and put into the evaluation mode
chpt_file_path  = os.path.join(CHECKPOITS_PATH, chpt_file_name)
model.load_state_dict(torch.load(chpt_file_path)['net'])

<All keys matched successfully>

In [6]:
time_taken = []

model.eval() # enter inference/evaluation mode 
for name in os.listdir(TEST_DATASET_PATH):
    path_img = os.path.join(TEST_DATASET_PATH, name)
        
    img = imageio.imread(path_img) / 255
    
    # record shape to revert to 
    H, W, _ = img.shape
    resize_back = transforms.Resize((H, W), interpolation=0)
    
    # convert to Tensors and fix the dimentions (Pytorch uses the channels in the first dimension)
    img = torch.FloatTensor(np.transpose(img, [2, 0 ,1])).unsqueeze(0) 
    
    # resize for the model
    img = resize(img)
    
    # put on the GPU
    img = img.to(DEVICE)
       
    # we do not need to calculate gradients
    with torch.no_grad():
        # Start time
        start_time = time.time()
        ## Prediction
        pred = model(img)
        # End timer
        end_time = time.time() - start_time

    time_taken.append(end_time)
    print("{} - {:.10f}".format(name, end_time))
        
    
    # resize back, nearest interpolation, since it's a mask
    pred = resize_back(pred)
    # put on cpu
    pred = pred.cpu()
        
    # remove channel: BATCH x 1 x H x W => BATCH x H x W
    pred = pred.squeeze(1)
        
    # comment out if your model contains a sigmoid or equivalent activation layer
    pred = torch.sigmoid(pred)
    # thresholding since that's how we will make predictions on new imputs
    pred = pred > 0.5 
    # remove BATCH => H x W
    pred = pred.squeeze(0)
    # convert to correct type
    pred = pred.numpy().astype(np.float32)
    # revert to standard intensities
    pred = pred * 255.0
    # save
    pred_path = os.path.join(MASK_PATH, name)
    cv2.imwrite(pred_path, pred)


mean_time_taken = np.mean(time_taken)
mean_fps = 1/mean_time_taken
print("\nMean FPS: ", mean_fps)

ckda5qdj900053a5sm6zg4bwb.jpg - 0.0089638233
ckda142i5000h3a5ssv9kauan.jpg - 0.0039613247
ckdaa12yj000r3a5srr4ahcg2.jpg - 0.0039975643
ckcaidxum2xrf0y6g0329hw80.jpg - 0.0039637089
ckda0kbde00083a5s2296juqy.jpg - 0.0042879581
ckda22o7b000v3a5s6zjzj7o3.jpg - 0.0042531490
ckda9x11x000h3a5st1g20mrf.jpg - 0.0038948059
ckda1jqsp000o3a5s5lh3pkdn.jpg - 0.0038816929
ckcbqi8ry27wq0y7m7xqmecor.jpg - 0.0038597584
ckd8sxdyn000a3b5sclphcakf.jpg - 0.0037970543
ckdaa001p000n3a5shzakc3ar.jpg - 0.0038394928
ckcbpz4vz28i70y5p347j9b4a.jpg - 0.0038213730
ckda1ehes000j3a5sr74zgivg.jpg - 0.0037589073
ckcbpmwgb267j0y5pcv7404dq.jpg - 0.0038518906
ckda1j0sy000n3a5stotfjqpc.jpg - 0.0038666725
ckcbsv8zd2i010y6g6te29eg7.jpg - 0.0038664341
ckcbt2pbv2ptl0y5p7xea6hg3.jpg - 0.0039634705
ckcbm3tjn0dyu0y4h4kbld9ro.jpg - 0.0038628578
ckdaa0oyd000q3a5sabbv22b0.jpg - 0.0038604736
ckcbqhgfw27qx0y7mbi0l9wn4.jpg - 0.0038928986
ckcbswp262i6n0y6g2krjfhdb.jpg - 0.0038199425
ckcbt0asc2k520y5n4e5i8c9y.jpg - 0.0038850307
ckda0hpik0